<a href="https://colab.research.google.com/github/sipocz/Zenga_DataScience/blob/main/Zenga_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q playwright nest_asyncio
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 15.1 MB/s eta 0:00:00
171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 20.0s171.6 MiB [] 0% 12.2s171.6 MiB [] 0% 6.7s171.6 MiB [] 1% 5.3s171.6 MiB [] 1% 5.0s171.6 MiB [] 1% 4.8s171.6 MiB [] 2% 4.7s171.6 MiB [] 2% 4.2s171.6 MiB [] 3% 4.0s171.6 MiB [] 3% 3.9s171.6 MiB [] 4% 3.7s171.6 MiB [] 5% 3.6s171.6 MiB [] 5% 3.5s171.6 MiB [] 5% 3.8s171.6 MiB [] 6% 3.7s171.6 MiB [] 6% 3.8s171.6 MiB [] 7% 3.8s171.6 MiB [] 7% 3.7s171.6 MiB [] 8% 3.5s171.6 MiB [] 9% 3.5s171.6 MiB [] 10% 3.4s171.6 MiB [] 10% 3.3s171.6 MiB [] 11% 3.2s171.6 MiB [] 13% 3.0s171.6 MiB [] 13% 2.9s171.6 MiB [] 14% 2.9s171.6 MiB [] 15% 2.7s171.6 MiB [] 16% 2.6s171.6 MiB [] 17% 2.5s171.6 MiB [] 18% 2.4s171.6 MiB [] 19% 2.3s171.6 MiB [] 21% 2.2s171.6 MiB [] 21% 2.1s171.6 MiB [] 23% 2.0s171.6 MiB [] 24% 2.0s171.6 MiB [] 25% 1.9s171.6 MiB [] 26% 1.8s171.6 MiB [] 26% 1.9s171.6 MiB [] 26% 2.0s171.6 MiB [] 29% 1.8s171.6 MiB [] 30% 1.7s171.6 MiB [] 32% 1.6s171.6 MiB [] 33% 1.6s171.6 MiB [] 34%

In [2]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_zenga_links():
    base_url = "https://www.zenga.hu"
    start_url = "https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=2"
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # <-- EZT JAVÍTOTTUK
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        ad_links = await page.query_selector_all("a[data-cy='advert-card-link']")
        print(f"Talált linkek száma: {len(ad_links)}")

        for a in ad_links:
            href = await a.get_attribute("href")
            if href:
                links.append(base_url + href)

        await browser.close()
        return links

# Futtatás
zenga_links = await scrape_zenga_links()

# Kiírás
for i, link in enumerate(zenga_links, 1):
    print(f"{i}. {link}")


Talált linkek száma: 20
1. https://www.zenga.hu/ingatlan/elado-csaladi-haz-babolna/8631699?page=2&pos=1&cr=31
2. https://www.zenga.hu/ingatlan/elado-csaladi-haz-sutto/8617466?page=2&pos=2&cr=31
3. https://www.zenga.hu/ingatlan/elado-ikerhaz-esztergom/8626642?page=2&pos=3&cr=31
4. https://www.zenga.hu/ingatlan/elado-csaladi-haz-tatabanya-banhida/8325162?page=2&pos=4&cr=31
5. https://www.zenga.hu/ingatlan/elado-csaladi-haz-tata/8622219?page=2&pos=5&cr=31
6. https://www.zenga.hu/ingatlan/elado-csaladi-haz-labatlan/8617873?page=2&pos=6&cr=31
7. https://www.zenga.hu/ingatlan/elado-csaladi-haz-uny/8618722?page=2&pos=7&cr=31
8. https://www.zenga.hu/ingatlan/elado-csaladi-haz-sarisap/8617736?page=2&pos=8&cr=31
9. https://www.zenga.hu/ingatlan/elado-csaladi-haz-domos/8618381?page=2&pos=9&cr=31
10. https://www.zenga.hu/ingatlan/elado-csaladi-haz-esztergom/8615925?page=2&pos=10&cr=31
11. https://www.zenga.hu/ingatlan/elado-csaladi-haz-annavolgy/8616546?page=2&pos=11&cr=31
12. https://www.zenga.hu

In [52]:
from playwright.async_api import async_playwright
import pandas as pd

# Az előzőleg megszerzett hirdetési linkek
links = [
    "https://www.zenga.hu/ingatlan/elado-csaladi-haz-babolna/8631699?page=2&pos=1&cr=31"

]

# Szövegek kigyűjtése a linkekről
async def scrape_ad_texts(links):
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        print(f"Scraping {len(links)} links...")
        for url in links:
            try:
                print(f"🔗 {url}")
                await page.goto(url, timeout=20000)
                try:
                    await page.get_by_text("Összes süti engedélyezése").click()
                    print("✅ Süti elfogadva")
                except:
                    print("ℹ️ Süti gomb nem jelent meg")


                await page.wait_for_timeout(4000)
                await page.screenshot(path="screenshot.png")
                desc_element = await page.query_selector("span[data-cy='advert-details-description']")
                text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
                print(f"✅ Szöveg: {text} ")
                results.append({"url": url, "text": text})
            except Exception as e:
                print(f"⚠️ Hiba: {str(e)}")
                results.append({"url": url, "text": f"Hiba: {str(e)}"})

        await browser.close()

    return pd.DataFrame(results)



#import ace_tools as tools; tools.display_dataframe_to_user(name="Hirdetés Szövegek", dataframe=df)



In [53]:
df = asyncio.run(scrape_ad_texts(links))

Scraping 1 links...
🔗 https://www.zenga.hu/ingatlan/elado-csaladi-haz-babolna/8631699?page=2&pos=1&cr=31
✅ Süti elfogadva
✅ Szöveg: ENNYIÉRT ITT CSAK LAKÁST KAPSZ,VELÜNK HÁZAD LESZ!FEDETT, TÉRKÖVEZETT GÉPKOCSI BEÁLLÓTERASZ, KERT, KLÍMAIsmerd meg Bábolna új utcájában épülő korszerű otthonodat! A magas műszaki tartalom és  a praktikus élettér tökéletes harmóniáját kínálja, ahol minden egyedi igényedre figyelmet fordítunk. Kérj bármit, mindenre lesz megoldásunk.MIT TARTALMAZ A FŰTÉSKÉSZ ÁR?-63 m² lakótér, amit jól átgondolt elrendezés tesz élhetővé.-10 m² fedett terasz, ahol minden reggel új élmény.-550 m² privát kert, ami tökéletes a szabadtéri programokhoz.-Térkövezett, részben fedett gépkocsibeálló és térkövezett járda a maximális kényelemért.-Két szoba + nappali, hogy mindenki megtalálja a helyét.-1 db hűtő-fűtő klíma a nappaliban a kellemes hőmérsékletért.-Tervezés, engedélyezési eljárás.FŐBB MŰSZAKI TARTALOM:-Főfalak 30 cm, közfalak 10 cm vastagságú téglából.-20 cm szigetelés-Kerámi

In [54]:
df

,url,text
0,https://www.zenga.hu/ingatlan/elado-csaladi-ha...,"ENNYIÉRT ITT CSAK LAKÁST KAPSZ,VELÜNK HÁZAD LE..."


In [55]:
df.text[0]

'ENNYIÉRT ITT CSAK LAKÁST KAPSZ,VELÜNK HÁZAD LESZ!FEDETT, TÉRKÖVEZETT GÉPKOCSI BEÁLLÓTERASZ, KERT, KLÍMAIsmerd meg Bábolna új utcájában épülő korszerű otthonodat! A magas műszaki tartalom és\xa0\xa0a praktikus élettér tökéletes harmóniáját kínálja, ahol minden egyedi igényedre figyelmet fordítunk. Kérj bármit, mindenre lesz megoldásunk.MIT TARTALMAZ A FŰTÉSKÉSZ ÁR?-63 m² lakótér, amit jól átgondolt elrendezés tesz élhetővé.-10 m² fedett terasz, ahol minden reggel új élmény.-550 m² privát kert, ami tökéletes a szabadtéri programokhoz.-Térkövezett, részben fedett gépkocsibeálló és térkövezett járda a maximális kényelemért.-Két szoba + nappali, hogy mindenki megtalálja a helyét.-1 db hűtő-fűtő klíma a nappaliban a kellemes hőmérsékletért.-Tervezés, engedélyezési eljárás.FŐBB MŰSZAKI TARTALOM:-Főfalak 30 cm, közfalak 10 cm vastagságú téglából.-20 cm szigetelés-Kerámia cserépfedés, színazonos bádogozással és ereszcsatornával.-Fehér színű 3 rétegű üvegezéssel ellátott műanyag nyílászárók red

In [43]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [56]:
szoveg=df.text[0]

In [66]:
prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": f"""
                            Értékeld az alábbi ingatlanhirdetés szöveget az alábbi szempontok alapján 1–5-ig,
                            és hozz létre egy összesített eredményt az 1-8 közötti értékek átlagaként ez legyen a 9. Összesítés :

                            1. Érthetőség
                            2. Részletesség
                            3. Szerkezet
                            4. Célcsoport megszólítása
                            5. Stílus és nyelvhelyesség
                            6. Előnyök kiemelése
                            7. Negatívumok őszinte kezelése
                            8. Vételre ösztönzés
                            9. Összesítés

Az értékelést json formában add vissza!

SZÖVEG:
{szoveg}
            """}
        ]


In [68]:
prompt_message


[{'role': 'system',
  'content': '\n                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.\n                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert. \n                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján. \n                    A szempontok:\n                        1. Érthetőség\n                        2. Részletesség / információtartalom\n                        3. Szerkezet, logikai felépítés\n                        4. Célcsoport megszólítása\n                        5. Stílus és nyelvhelyesség\n                        6. Előnyök kiemelése\n                        7. Negatívumok őszinte kezelése\n                        8. Eladásra ösztönzés'},
 {'role': 'user',
  'content': '\n                            Értékeld az alábbi ingatlanhirdetés szöveget az alábbi szempontok alapján 1–5-ig, \n      

In [67]:
from openai import OpenAI

client = OpenAI(api_key=key)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=prompt_message,
    temperature=0.2,
    max_tokens=200
)

print(response.choices[0].message.content)


{
    "értékelés": {
        "1. Érthetőség": 3,
        "2. Részletesség": 4,
        "3. Szerkezet": 3,
        "4. Célcsoport megszólítása": 4,
        "5. Stílus és nyelvhelyesség": 2,
        "6. Előnyök kiemelése": 5,
        "7. Negatívumok őszinte kezelése": 3,
        "8. Vételre ösztönzés": 4,
        "9. Összesítés": 3.5
    }
}


In [71]:
response.choices[0].message.content

'{\n    "értékelés": {\n        "1. Érthetőség": 3,\n        "2. Részletesség": 4,\n        "3. Szerkezet": 3,\n        "4. Célcsoport megszólítása": 4,\n        "5. Stílus és nyelvhelyesség": 2,\n        "6. Előnyök kiemelése": 5,\n        "7. Negatívumok őszinte kezelése": 3,\n        "8. Vételre ösztönzés": 4,\n        "9. Összesítés": 3.5\n    }\n}'

In [ ]:
resp_token=response.choices[0]

In [73]:
import json
out_dict= json.loads(response.choices[0].message.content)

In [74]:
out_dict

{'értékelés': {'1. Érthetőség': 3,
  '2. Részletesség': 4,
  '3. Szerkezet': 3,
  '4. Célcsoport megszólítása': 4,
  '5. Stílus és nyelvhelyesség': 2,
  '6. Előnyök kiemelése': 5,
  '7. Negatívumok őszinte kezelése': 3,
  '8. Vételre ösztönzés': 4,
  '9. Összesítés': 3.5}}